In [8]:
import androidx.lifecycle.ViewModel
import androidx.lifecycle.viewModelScope
import co.touchlab.kermit.Logger
import kotlinx.coroutines.*
import kotlinx.coroutines.flow.*

interface ViewEvent
interface ViewState
interface ViewSideEffect

abstract class MVIViewModel<Event : ViewEvent, UiState : ViewState, Effect : ViewSideEffect> :
    ViewModel() {

    private val initialState: UiState by lazy { setInitialState() }
    abstract fun setInitialState(): UiState

    /**
     * By default, we don't wait for anything before onStart.
     * This makes it optional for ViewModels that do not require readiness checks.
     * ViewModels that need to wait can override this function.
     */
    protected open suspend fun awaitReadiness(): Boolean = true

    /**
     * Default timeout for readiness checks in milliseconds.
     * ViewModels can override this if they need a different timeout.
     */
    protected open val readinessTimeoutMillis: Long = 2000L

    private val _viewState: MutableStateFlow<UiState> = MutableStateFlow(initialState)
    val viewState: StateFlow<UiState> = _viewState.onStart {
        val ready = awaitReadinessWithTimeout()
        if (ready) {
            onStart()
        } else {
            handleReadinessFailed()
        }
    }.stateIn(
        scope = viewModelScope,
        started = SharingStarted.WhileSubscribed(5000),
        initialValue = initialState
    )

    abstract fun onStart()

    private val _event: MutableSharedFlow<Event> = MutableSharedFlow()

    private val _effect: MutableSharedFlow<Effect> = MutableSharedFlow()
    val effect: SharedFlow<Effect> = _effect

    init {
        subscribeToEvents()
    }

    fun setEvent(event: Event) {
        viewModelScope.launch { _event.emit(event) }
    }

    protected fun setState(reducer: UiState.() -> UiState) {
        _viewState.update { currentState -> currentState.reducer() }
    }

    private fun subscribeToEvents() {
        viewModelScope.launch {
            _event.collect {
                handleEvents(it)
            }
        }
    }

    abstract fun handleEvents(event: Event)

    protected fun setEffect(builder: () -> Effect) {
        val effectValue = builder()
        viewModelScope.launch { _effect.emit(effectValue) }
    }

    /**
     * If awaitReadiness() returns false, handle that scenario here.
     * For example, show an error message or navigate away.
     */
    protected open fun handleReadinessFailed() {
        Logger.e("MVIViewModel: Readiness check failed")
        println("MVIViewModel: Readiness check failed")
        // By default, do nothing. Concrete ViewModels can override if needed.
        // Example:
        // setState { copy(errorMessage = "Initialization failed") }
        // setEffect { MyEffect.ShowError("Initialization failed") }
    }

    /**
     * Internal function to handle timeout in readiness checks.
     */
    private suspend fun awaitReadinessWithTimeout(): Boolean {
        return withTimeoutOrNull(readinessTimeoutMillis) {
            awaitReadiness()
        } ?: false
    }
}

In [ ]:
interface SessionIds

In [10]:

import androidx.lifecycle.ViewModel
import androidx.lifecycle.viewModelScope
import co.touchlab.kermit.Logger
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.MutableStateFlow
import kotlinx.coroutines.flow.StateFlow
import kotlinx.coroutines.flow.asStateFlow
import kotlinx.coroutines.flow.collectLatest
import kotlinx.coroutines.flow.update
import kotlinx.coroutines.launch

interface SessionHandlerDelegate<Session : SessionIds> {
    val sessionState: StateFlow<Session>
}

class SessionHandler<Session : SessionIds>(
    initialSession: Session,
    private val sessionFlow: Flow<Session>,
) : ViewModel(), SessionHandlerDelegate<Session> {
    // TODO: When koin allows for viewModelScope injection remove ViewModel() inheritance

    private val _sessionState: MutableStateFlow<Session> = MutableStateFlow(initialSession)
    override val sessionState: StateFlow<Session> = _sessionState.asStateFlow()

    init {
        viewModelScope.launch {
            sessionFlow.collectLatest { session ->
                Logger.withTag("SessionHandler").d { session.toString() }
                _sessionState.update { session }
            }
        }
    }
}

In [ ]:
package org.example.presentation.screen.simple

import androidx.lifecycle.viewModelScope
import kotlinx.coroutines.flow.collectLatest
import kotlinx.coroutines.flow.first
import kotlinx.coroutines.launch
import org.example.common.presentation.MVIViewModel
import org.example.common.presentation.SessionHandler
import org.example.common.presentation.SessionHandlerDelegate
import org.example.domain.usecase.GetItemsUseCase
import org.example.domain.usecase.UpdateItemUseCase
import org.example.domain.model.SessionIds
import org.example.shared.value.ItemId

class SimpleScreenViewModel(
    private val getItemsUseCase: GetItemsUseCase,
    private val updateItemUseCase: UpdateItemUseCase,
    private val getSessionIdsFlowUseCase: GetSessionIdsFlowUseCase
) : MVIViewModel<
        SimpleScreenContract.Event,
        SimpleScreenContract.State,
        SimpleScreenContract.Effect>(),
    SessionHandlerDelegate<SessionIds> by SessionHandler(
        initialSession = SessionIds(),
        sessionFlow = getSessionIdsFlowUseCase()
    ) {

    override fun setInitialState(): SimpleScreenContract.State {
        return SimpleScreenContract.State(
            items = emptyList(),
            isLoading = false,
            errorMessage = null
        )
    }

    override suspend fun awaitReadiness(): Boolean {
        val sessionIds = sessionState.first { it.userId != null }
        return sessionIds.userId != null
    }

    override fun handleReadinessFailed() {
        setState { copy(errorMessage = "Session not ready") }
    }

    override fun onStart() {
        viewModelScope.launch {
            collectItems()
        }
    }

    // TABLE OF CONTENTS - All possible events handled here
    override fun handleEvents(event: SimpleScreenContract.Event) {
        when (event) {
            is SimpleScreenContract.Event.LoadItems -> {
                loadItems()
            }

            is SimpleScreenContract.Event.SelectItem -> {
                selectItem(event.itemId)
            }

            is SimpleScreenContract.Event.UpdateItem -> {
                viewModelScope.launch {
                    updateItem(
                        itemId = event.itemId,
                        newValue = event.newValue,
                        onSuccess = {
                            showSnackbar("Item updated successfully")
                        },
                        onError = { errorMessage ->
                            showSnackbar("Failed to update item: $errorMessage")
                        }
                    )
                }
            }

            is SimpleScreenContract.Event.Navigation.Back -> {
                navigateBack()
            }

            is SimpleScreenContract.Event.ShowDetails -> {
                showItemDetails(event.itemId)
            }

            is SimpleScreenContract.Event.RefreshItems -> {
                viewModelScope.launch {
                    fetchItemsFromRepository()
                }
            }
        }
    }

    private fun loadItems() {
        setState { copy(isLoading = true) }
        // Note: No viewModelScope.launch here - this would be called from handleEvents if needed
    }

    private fun selectItem(itemId: ItemId) {
        setState {
            copy(
                items = items.map { item ->
                    if (item.id == itemId) {
                        item.copy(isSelected = !item.isSelected)
                    } else item
                }
            )
        }
    }

    private suspend fun updateItem(
        itemId: ItemId,
        newValue: String,
        onSuccess: () -> Unit,
        onError: (String) -> Unit
    ) {
        val userId = sessionState.value.userId.handleNull() ?: return

        when (val result = updateItemUseCase(userId, itemId, newValue)) {
            is UpdateItemUseCase.Result.Success -> {
                onSuccess()
            }
            is UpdateItemUseCase.Result.Error -> {
                onError(result.message)
            }
        }
    }

    private fun navigateBack() {
        setEffect { SimpleScreenContract.Effect.Navigation.Back }
    }

    private fun showItemDetails(itemId: ItemId) {
        setEffect { SimpleScreenContract.Effect.Navigation.ToDetails(itemId) }
    }

    private suspend fun collectItems() {
        getItemsUseCase()
            .collectLatest { result ->
                when (result) {
                    is GetItemsUseCase.Result.Success -> {
                        setState {
                            copy(
                                items = result.items.toDisplayModel(),
                                isLoading = false
                            )
                        }
                    }
                    is GetItemsUseCase.Result.Error -> {
                        setState {
                            copy(
                                items = emptyList(),
                                isLoading = false,
                                errorMessage = result.message
                            )
                        }
                    }
                }
            }
    }

    private suspend fun fetchItemsFromRepository() {
        // Implementation details...
        // This function is suspend but doesn't launch coroutines itself
        // It's called from within viewModelScope.launch in handleEvents
    }

    private fun showSnackbar(message: String) {
        setEffect {
            SimpleScreenContract.Effect.ShowSnackBar(message)
        }
    }

    private fun String?.handleNull(): String? {
        if (this == null) {
            setState { copy(errorMessage = "Required value is null") }
        }
        return this
    }
}

### Corrected Simplified Example

```kotlin

```

### Updated Coding Conventions and Patterns Documentation

#### ### Coroutine Management Rules

**viewModelScope.launch Restriction**
- `viewModelScope.launch` should ONLY be called from:
  - `handleEvents()` function
  - `onStart()` function
- Never call `viewModelScope.launch` from private functions

**Lambda Pattern for Async Operations**
- When private functions need coroutine execution, use lambda parameters
- Pass success/error (as an example) callbacks from `handleEvents()`
- Execute the lambdas within the `viewModelScope.launch` block in `handleEvents()`

**Example Pattern:**
```kotlin
// In handleEvents()
is Event.UpdateItem -> {
    viewModelScope.launch {
        updateItem(
            itemId = event.itemId,
            onSuccess = { showSnackbar("Success") },
            onError = { error -> showSnackbar("Error: $error") }
        )
    }
}

// Private function with lambda parameters
private suspend fun updateItem(
    itemId: ItemId,
    onSuccess: () -> Unit,
    onError: (String) -> Unit
) {
    // Implementation
    when (result) {
        is Success -> onSuccess()
        is Error -> onError(result.message)
    }
}
```

#### ### Function Organization Patterns

**handleEvents() as Coroutine Controller**
- Central place for all `viewModelScope.launch` calls
- Controls when and how coroutines are started
- Manages callback execution for async operations


#### ### Benefits of This Pattern

**Centralized Coroutine Management**
- All coroutine launches are visible in one place (`handleEvents()` and `onStart()`)
- Easier to track and debug coroutine execution
- Prevents scattered coroutine launches throughout the codebase

**Clear Control Flow**
- Easy to see what happens after async operations complete
- Callbacks are defined at the call site in `handleEvents()`
- No hidden coroutine launches in private functions

**Maintainability**
- Consistent pattern across all ViewModels
- Easy to add new async operations by following the same pattern
- Clear separation between sync and async operations